In [1]:
import sys
sys.path.append("./learning-tsp/src")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install tqdm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
from problems.tsp.problem_tsp import TSP
from utils import load_model, move_to
from torch.utils.data import DataLoader
import boto3
import sagemaker

In [5]:
from inference import *

In [6]:
import pandas as pd

In [7]:
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

In [8]:
session = sagemaker.Session()
BUCKET = session.default_bucket()                   # Set a default S3 bucket

In [9]:
# set USE_PRETRAINED_MODEL to False if you have trained a model using pytorch_smdataparallel_tsp_demo.ipynb
USE_PRETRAINED_MODEL = True

In [10]:
PRETRAINED_MODEL_PATH = "learning-tsp/pretrained/tsp_20-50/rl-ar-var-20pnn-gnn-max_20200313T002243/model.tar.gz"

# 1. Test inference code locally

## Prepare data

In [11]:
dataset_path = None
batch_size = 1
accumulation_steps = 80
num_samples = 2 # 1280 samples per TSP size 

neighbors = 0.20
knn_strat = 'percentage'

In [12]:
dataset = TSP.make_dataset(
    filename=dataset_path,
    batch_size=batch_size,
    num_samples=num_samples,
    min_size=10,
    max_size=10,
    neighbors=neighbors,
    knn_strat=knn_strat,
    supervised=False
)
dataloader = DataLoader(dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=0)


Generating 2 samples of TSP10-10...


100%|##########| 2/2 [00:00<00:00, 996.27it/s]


In [13]:
# transform data
data = []
for bat_idx, bat in enumerate(dataloader):
    input = {}
    input["nodes"] = bat["nodes"].tolist()
    data.append(input)
for record in data:
    record["neighbors"] = neighbors

In [14]:
data

[{'nodes': [[[0.17237919569015503, 0.30322661995887756],
    [0.4001781642436981, 0.08303876966238022],
    [0.9577174782752991, 0.06723511219024658],
    [0.3205690085887909, 0.864788293838501],
    [0.7107783555984497, 0.4769577980041504],
    [0.2889561057090759, 0.23669394850730896],
    [0.25502970814704895, 0.6648805141448975],
    [0.7122745513916016, 0.8629788160324097],
    [0.15989430248737335, 0.5915039777755737],
    [0.6722036600112915, 0.42656928300857544]]],
  'neighbors': 0.2},
 {'nodes': [[[0.016316097229719162, 0.9005574584007263],
    [0.5326317548751831, 0.3426164388656616],
    [0.7196614146232605, 0.14507879316806793],
    [0.47778037190437317, 0.18811026215553284],
    [0.8428910970687866, 0.8391184210777283],
    [0.5771450400352478, 0.7313387989997864],
    [0.35447993874549866, 0.8077568411827087],
    [0.9068283438682556, 0.30471447110176086],
    [0.05323702096939087, 0.22702862322330475],
    [0.0816958099603653, 0.9146664142608643]]],
  'neighbors': 0.2}]

## Prepare the model

In [15]:
# Getting the latest model data from the training jobs
def get_latest_model():
    client = boto3.client("sagemaker")

    # Get the trained sklearn model
    response = client.list_training_jobs(
        NameContains="pytorch-smdataparallel-tsp",
        StatusEquals="Completed",
        SortBy="CreationTime",
        SortOrder="Descending",
    )
    training_job_name = response["TrainingJobSummaries"][0]["TrainingJobName"]
    model_s3 = client.describe_training_job(TrainingJobName=training_job_name)[
        "ModelArtifacts"
    ]["S3ModelArtifacts"]
    return model_s3

# Upload a pretrained model to s3
def upload_pretrained_model():
    s3 = boto3.resource('s3')
    s3.meta.client.upload_file(PRETRAINED_MODEL_PATH,
                               BUCKET,
                               PRETRAINED_MODEL_PATH)
    return f"s3://{BUCKET}/{PRETRAINED_MODEL_PATH}"

In [16]:
if USE_PRETRAINED_MODEL == True:
    model_data = upload_pretrained_model()
else:
    model_data = get_latest_model()

In [17]:
model_data

's3://sagemaker-ap-southeast-2-161422014849/pretrained/tsp_20-50/rl-ar-var-20pnn-gnn-max_20200313T002243/model.tar.gz'

## Download the model locally for testing

In [18]:
!aws s3 cp $model_data ./

download: s3://sagemaker-ap-southeast-2-161422014849/pretrained/tsp_20-50/rl-ar-var-20pnn-gnn-max_20200313T002243/model.tar.gz to ./model.tar.gz


In [19]:
!mkdir -p model

In [20]:
!tar -xvzf ./model.tar.gz -C ./model/

args.json
model.pt


In [21]:
!ls model

args.json  model.pt


## Load model

In [22]:
model_dir = "./model"
model = model_fn(model_dir)

./model/model.pt
./model
./model/model.pt

Loading model from ./model/model.pt


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'nets.attention_model.AttentionModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.E

In [23]:
model

AttentionModel(
  (init_embed): Linear(in_features=2, out_features=128, bias=True)
  (embedder): GNNEncoder(
    (init_embed_edges): Embedding(2, 128)
    (layers): ModuleList(
      (0): GNNLayer(
        (U): Linear(in_features=128, out_features=128, bias=True)
        (V): Linear(in_features=128, out_features=128, bias=True)
        (A): Linear(in_features=128, out_features=128, bias=True)
        (B): Linear(in_features=128, out_features=128, bias=True)
        (C): Linear(in_features=128, out_features=128, bias=True)
        (norm_h): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (norm_e): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      )
      (1): GNNLayer(
        (U): Linear(in_features=128, out_features=128, bias=True)
        (V): Linear(in_features=128, out_features=128, bias=True)
        (A): Linear(in_features=128, out_features=128, bias=True)
        (B): Linear(in_features=128, out_feat

## input

In [24]:
serializer = JSONLinesSerializer()

In [25]:
data_jsonlines = serializer.serialize(data)

In [30]:
#request_body = serializer.serialize(data_jsonlines).encode("utf-8")

In [26]:
request_body = data_jsonlines.encode("utf-8")

In [27]:
input_data = input_fn(request_body)

request received!!
<class 'bytes'>
2
<class 'str'> 458
<class 'str'> 460


In [28]:
with open("inference_input", 'w') as file:
    file.write(data_jsonlines)

In [29]:
# upload to S3 for batch transform
!aws s3 cp inference_input s3://$BUCKET/data/inference/

upload: ./inference_input to s3://sagemaker-ap-southeast-2-161422014849/data/inference/inference_input


## Prediction

In [30]:
input_data

[{'nodes': [[[0.17237919569015503, 0.30322661995887756],
    [0.4001781642436981, 0.08303876966238022],
    [0.9577174782752991, 0.06723511219024658],
    [0.3205690085887909, 0.864788293838501],
    [0.7107783555984497, 0.4769577980041504],
    [0.2889561057090759, 0.23669394850730896],
    [0.25502970814704895, 0.6648805141448975],
    [0.7122745513916016, 0.8629788160324097],
    [0.15989430248737335, 0.5915039777755737],
    [0.6722036600112915, 0.42656928300857544]]],
  'neighbors': 0.2},
 {'nodes': [[[0.016316097229719162, 0.9005574584007263],
    [0.5326317548751831, 0.3426164388656616],
    [0.7196614146232605, 0.14507879316806793],
    [0.47778037190437317, 0.18811026215553284],
    [0.8428910970687866, 0.8391184210777283],
    [0.5771450400352478, 0.7313387989997864],
    [0.35447993874549866, 0.8077568411827087],
    [0.9068283438682556, 0.30471447110176086],
    [0.05323702096939087, 0.22702862322330475],
    [0.0816958099603653, 0.9146664142608643]]],
  'neighbors': 0.2}]

In [31]:
prediction = predict_fn(input_data, model)

  0%|          | 0/2 [00:00<?, ?it/s]./src/nets/encoders/gnn_encoder.py:118: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/aten/src/ATen/native/IndexingUtils.h:25.)
  Vh[graph.unsqueeze(-1).expand_as(Vh)] = 0
./src/nets/attention_model.py:573: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/aten/src/ATen/native/IndexingUtils.h:25.)
  compatibility[mask[None, :, :, None, :].expand_as(compatibility)] = -1e10
./src/nets/attention_model.py:597: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370116979/work/aten/src/ATen/native/IndexingUtils.h:25.)
  logits[mask] = -1e10
100%|██████████| 2/2 [00:00<00:00,  9.91it/s]

cost:tensor([2.8009])
cost:tensor([3.2031])


In [32]:
prediction

[[[7, 4, 9, 2, 1, 5, 0, 8, 6, 3]], [[4, 7, 2, 1, 3, 8, 0, 9, 6, 5]]]

## output

In [33]:
output = output_fn(prediction)

In [34]:
output

('[[7, 4, 9, 2, 1, 5, 0, 8, 6, 3]]\n[[4, 7, 2, 1, 3, 8, 0, 9, 6, 5]]\n',
 'application/jsonlines')

In [35]:
with open("prediction", 'w') as file:
    file.write(output[0])

# 2. Test inference code via endpoints

In [40]:
import sagemaker

role = sagemaker.get_execution_role()

from sagemaker.pytorch import PyTorchModel

model_sm = PyTorchModel(
    model_data=model_data,
    source_dir="learning-tsp/src",
    entry_point="inference.py",
    role=role,
    framework_version="1.8.1",
    py_version="py36",
)

In [42]:
predictor = model_sm.deploy(initial_instance_count=1,
                         instance_type="ml.m4.xlarge",
                         serializer=JSONLinesSerializer(),
                         deserializer=JSONLinesDeserializer(),)

---------------!

In [43]:
# Send the sampled images to endpoint for inference
prediction = predictor.predict(data)

In [44]:
prediction

[[[5, 8, 0, 1, 6, 4, 9, 7, 3, 2]], [[9, 7, 4, 5, 0, 1, 6, 2, 3, 8]]]

# 3. Batch Transform

In [45]:
transformer = model_sm.transformer(instance_count=1, 
                     instance_type='ml.m5.2xlarge', 
                     strategy='MultiRecord',
                     assemble_with='Line',
                     accept='application/jsonlines',
                     max_concurrent_transforms=4,
                     env = {'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '3600' },
                     #output_path='s3://{}/output'.format(bucket),
                     #sagemaker_session=sagemaker_session
                                  )

In [46]:
transformer.transform(f's3://{BUCKET}/data/inference/inference_input',
                      content_type='application/jsonlines',
                     split_type='Line',
                     wait=False,)

# 4. Clean up (Optional)

In [ ]:
## Delete the SageMaker endpoint
#predictor.delete_endpoint()
#
## Delete the SageMaker model
#model_sm.delete_model()